In [1]:
import pickle
import pandas as pd
import numpy as np
import nrrd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output

from radiomics import featureextractor, getTestCase
import SimpleITK as sitk
import subprocess

import os
import glob
import shutil

import time
import datetime

print(datetime.datetime.now())

2022-02-22 18:43:12.597307


In [2]:
path_data = '../data/d20220201/'

# INPUTS
fn_patient = '../metadata/Celine_FINAL_RADIOPOP_DATA_with LRE Death_220126.xlsx'
fn_patient2 = '../metadata/List Celine Radipop 20.12.21.xlsx'
path_masks = path_data + '0raw_masks/'
path_dcmzips = path_data + '0raw_dicom/'

# TEMP PATHS
temp_unzip = path_data + 'temp_unzip/'

# OUTPUTS
path_nrrds = path_data + 'nrrds_metadata/'

In [3]:
# prepare directories
try:
    os.mkdir(path_nrrds)
except OSError:
    print ("Creation of the directory %s failed" % path_nrrds)
else:
    print ("Successfully created the directory %s " % path_nrrds)

Creation of the directory data/d20220201/nrrds_metadata/ failed


In [4]:
patients_df = pd.read_excel(fn_patient, index_col=0, dtype=str)#, index_col=0, header=1, nrows=371, dtype=str)
patients_df['patientID'] = patients_df.index
patients_df['patientID'] = patients_df['patientID'].apply(lambda x: str(x).zfill(4))
patients_df = patients_df.set_index('patientID')
patients_df

patients_df2 = pd.read_excel(fn_patient2, index_col=0, dtype=str)#, index_col=0, header=1, nrows=371, dtype=str)
patients_df2['temp'] = patients_df2.index
patients_df2['patientID'] = patients_df2['temp'].str.split(" ", n=1).str[0]
patients_df2.patientID.fillna(patients_df2.temp, inplace=True)
patients_df2 = patients_df2[patients_df2['patientID'].astype(int) >= 1000]
patients_df2 = patients_df2.set_index('patientID')
patients_df2

listPatients = patients_df.index.tolist() + patients_df2.index.tolist()
print(len(listPatients))

562


In [5]:
t0 = time.time()

#listMasks = [x for x in os.listdir(path_masks) if os.path.isdir(os.path.join(path_masks, x))]
listConvSuccess = []
listNoDcm = []
listConvError = []
unzipError = []
checkTree = []


# for each available mask, check for raw image zip
for eaPatient in listPatients:
    patientID = str(eaPatient)
    fn_dcmzip = path_dcmzips + str(patientID) + '.zip'
    if os.path.exists(fn_dcmzip):
        path_nrrdTarget = path_nrrds + str(patientID) + '.nrrd'
        if os.path.exists(path_nrrdTarget):
            print(patientID + ': nrrd already complete')
            listConvSuccess.append(patientID)
        else:
            os.mkdir(temp_unzip)
            try:
                t1 = time.time()
                # unzip raw image file
                cmd = 'unzip %s -d %s' %(fn_dcmzip, temp_unzip)   # unzip WHAT -d WHERE
                subprocess.run(cmd, shell=True)
                try:
                    # identify signs of hidden folders
                    garbageIndicators = ["__MACOSX", "/."]

                    # use os.walk to get names of all directories
                    listDir = []
                    for root, dirs, files in os.walk(temp_unzip, topdown=False):
                        for name in [x for x in dirs]:
                            listDir.append(os.path.join(root, name))
                    # remove directories that are hidden folders
                    notHiddenDir = [s for s in listDir if not any(x in s for x in garbageIndicators)]
                    # check if dicom files are in deepest directory
                    potentialDir = max(notHiddenDir)
                    potentialFiles = [s for s in os.listdir(potentialDir) if '.dcm' in s and not s.startswith('.')]
                    if potentialFiles: # if potentialFiles is not empty
                        path_dcm = potentialDir
                        # convert dicom to nrrd
                        cmd = "plastimatch convert --input %s --output-img %s"%(path_dcm, path_nrrdTarget)
                        output = subprocess.run(cmd, shell=True, capture_output=True)
                        if not os.path.exists(path_nrrdTarget):
                            print(patientID + ': not successful ' + str(time.time() - t1) + ' seconds.')
                            print(output)
                            listConvError.append(patientID)
                        else:
                            print(patientID + ': converted, ' + str(time.time() - t1) + ' seconds.')
                            listConvSuccess.append(patientID)
                    else:
                        checkTree.append()
                except:
                    print(str(patientID) + ': nrrd conversion error')
                    print(cmd)
                    listConvError.append(patientID)
            except:
                print(str(patientID) + ': unzip error')
                unzipError.append(patientID)
            # remove temp unzip
            shutil.rmtree(temp_unzip)
    else:
        print(str(patientID) + ': dcm files missing')
        listNoDcm.append(patientID)

0001: nrrd already complete
0002: nrrd already complete
0004: nrrd already complete
0005: nrrd already complete
0006: nrrd already complete
0007: nrrd already complete
0008: nrrd already complete
0009: nrrd already complete
0010: nrrd already complete
0011: nrrd already complete
0013: nrrd already complete
0015: nrrd already complete
0016: nrrd already complete
0018: nrrd already complete
0019: nrrd already complete
0020: nrrd already complete
0021: nrrd already complete
0022: nrrd already complete
0023: nrrd already complete
0024: nrrd already complete
0025: nrrd already complete
0026: nrrd already complete
0027: nrrd already complete
0028: nrrd already complete
0029: nrrd already complete
0030: nrrd already complete
0031: nrrd already complete
0032: nrrd already complete
0033: nrrd already complete
0034: nrrd already complete
0035: nrrd already complete
0036: nrrd already complete
0037: nrrd already complete
0038: nrrd already complete
0039: nrrd already complete
0040: nrrd already c

In [6]:
print('Problem with conversion')
print(listConvError)

print('No raw files:')
print(listNoDcm)

Problem with conversion
[]
No raw files:
[]


In [7]:
t0 = time.time()

listConvError2 = []
problems = listConvError

# for each available mask, check for raw image zip
for eaPatient in problems:
    patientID = eaPatient
    fn_dcmzip = path_dcmzips + str(patientID) + '.zip'
    if os.path.exists(fn_dcmzip):
        path_nrrdTarget = path_nrrds + str(patientID) + '.nrrd'
        if os.path.exists(path_nrrdTarget):
            print(str(patientID) + ': nrrd already complete')
            listConvSuccess.append(patientID)
        else:
            os.mkdir(temp_unzip)
            try:
                t1 = time.time()
                # unzip raw image file
                cmd = 'unzip %s -d %s' %(fn_dcmzip, temp_unzip)   # unzip WHAT -d WHERE
                subprocess.run(cmd, shell=True)
                try:
                    potentialDir = temp_unzip
                    potentialFiles = [s for s in os.listdir(potentialDir) if '.dcm' in s and not s.startswith('.')]
                    if potentialFiles: # if potentialFiles is not empty
                        path_dcm = potentialDir
                        # convert dicom to nrrd
                        cmd = "plastimatch convert --input %s --output-img %s"%(path_dcm, path_nrrdTarget)
                        output = subprocess.run(cmd, shell=True, capture_output=True)
                        # remove temp_unzip files
                        
                        if not os.path.exists(path_nrrdTarget):
                            print(patientID + ': not successful ' + str(time.time() - t1) + ' seconds.')
                            print(output)
                            listConvError2.append(patientID)
                        else:
                            print(patientID + ': converted, ' + str(time.time() - t1) + ' seconds.')
                            listConvSuccess.append(patientID)
                    else:
                        checkTree.append()
                except:
                    print(str(patientID) + ': nrrd conversion error')
                    print(cmd)
                    listConvError2.append(patientID)
            except:
                print(str(patientID) + ': unzip error')
                unzipError.append(patientID)
            # remove temp unzip
            shutil.rmtree(temp_unzip)
    else:
        print(str(patientID) + ': dcm files missing')
        listNoDcm.append(patientID)

In [8]:
# # identify signs of hidden folders
# garbageIndicators = ["__MACOSX", "/."]

# # use os.walk to get names of all directories
# listDir = []
# for root, dirs, files in os.walk(temp_unzip, topdown=False):
#     for name in [x for x in dirs]:
#         listDir.append(os.path.join(root, name))
# # remove directories that are hidden folders
# notHiddenDir = [s for s in listDir if not any(x in s for x in garbageIndicators)]
# # check if dicom files are in deepest directory
# potentialDir = max(notHiddenDir)
# potentialFiles = [s for s in os.listdir(potentialDir) if '.dcm' in s and not s.startswith('.')]

# if potentialFiles: # if potentialFiles is not empty
#     path_dcm = potentialDir
# else:
#     stillStillStillConvError.append()

In [9]:
print(datetime.datetime.now())

2022-02-22 18:43:15.197582
